In [1]:
import sys
import os
print("insert absolute path to where utils folder is located, e.g. /Users/docs/projects/datatonic-challenge")
path_to_utils = input()
sys.path.insert(0, os.path.abspath(path_to_utils + '/utils')) #point this to the where util is relatively to your working directory
from util import *

insert absolute path to where utils folder is located, e.g. /Users/docs/projects/datatonic-challenge
/Users/novikov/python_notebooks/datatonic-challenge


In [91]:
import pandas as pd
pd.set_option('display.max_columns', None) #will show all columns, comment out if not interested, as it may make the notebook slow
import numpy as np
import json 
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from sklearn.svm import SVR
from sklearn.linear_model import Ridge
from sklearn.metrics import r2_score
from sklearn.ensemble import RandomForestRegressor

In [41]:
data_dir = get_path_to_data_dir()
movie_data = pd.read_pickle(data_dir + 'pre-processed/movie_encoded_only_numeric.pkl')
movie_data = movie_data.drop(['id', 'vote_average', 'vote_count'], axis = 'columns') #dropping features which won't be available before movie release
print(type(movie_data['budget']))
movie_data.head()


<class 'pandas.core.series.Series'>


budget    revenue  runtime  release_date  lead_male  lead_female  \
0  105000000  351040419    143.0           130          1            0   
1  150000000  335154643    116.0           139          1            0   
2  140000000  258022233     91.0           220          1            0   
3  145000000  401128639    112.0           183          1            0   
4   15000000   77000000    120.0           211          1            0   

   lead2_male  lead2_female  Action  Adventure  Animation  Comedy  Crime  \
0           1             0       0          0          0       0      0   
1           0             1       1          1          0       0      0   
2           1             0       1          0          0       1      1   
3           1             0       1          1          0       0      0   
4           1             0       1          1          0       0      0   

   Documentary  Drama  Family  Fantasy  Foreign  History  Horror  Music  \
0            0      1       0        0        0        0       0      0   
1            0      0       0        1        0        0       0      0   
2            0      0       0        0        0        0       0      0   
3            0      0       0        1        0        0       0      0   
4            0      1       0        0        0        0       1      0   

   Mystery  Romance  Science Fiction  TV Movie  Thriller  War  Western  1960s  \
0        0        1                0         0         0    0        0      0   
1        0        1                0         0         0    0        0      0   
2        0        0                0         0         1    0        0      0   
3        0        0                0         0         0    0        0      0   
4        0        0                1         0         0    0        0      0   

   1970s  1980s  19th century  3d  adoption  adultery  adventure  \
0      0      0             0   1         0         0          0   
1      0      0             0   0         0         0          0   
2      0      0             0   0         0         0          0   
3      0      0             0   0         0         0          0   
4      0      0             0   0         0         0          0   

   aftercreditsstinger  afterlife  agent  airplane  airport  alcohol  \
0                    0          0      0         0        0        0   
1                    0          0      0         0        0        0   
2                    0          0      0         0        0        0   
3                    0          0      0         0        0        0   
4                    0          0      0         0        0        0   

   alcoholic  alcoholism  alien  alien invasion  american football  amnesia  \
0          0           0      0               0                  0        0   
1          0           0      0               0                  0        0   
2          0           0      0               0                  0        0   
3          0           0      0               0                  0        0   
4          0           0      0               0                  0        0   

   android  animation  anti hero  army  artificial intelligence  assassin  \
0        0          0          0     0                        0         0   
1        0          0          0     0                        0         0   
2        0          0          0     0                        0         0   
3        0          0          0     0                        0         0   
4        0          0          0     0                        0         0   

   assassination  astronaut  australia  author  baby  bank  bar  baseball  \
0              0          0          0       0     0     0    0         0   
1              0          0          0       0     0     0    0         0   
2              0          0          0       0     0     0    0         0   
3              0          0          0       0     0     0    0         0   
4

In [42]:
'''

def normalize_cube_root(y_cube):
    movie_data.budget**(1/3)
    y_cube = normalize(y_log)
    return y_log_norm

def feature_scaling(column_name, df):
    data_log = np.log(movie_data[str(column_name)] + 1)
    data_log_norm = normalize_log(data_log)
    df[column_name] = data_log_norm
    return df'''

'\n\ndef normalize_cube_root(y_cube):\n    movie_data.budget**(1/3)\n    y_cube = normalize(y_log)\n    return y_log_norm\n\ndef feature_scaling(column_name, df):\n    data_log = np.log(movie_data[str(column_name)] + 1)\n    data_log_norm = normalize_log(data_log)\n    df[column_name] = data_log_norm\n    return df'

In [43]:


#minmax = MinMaxScaler()
#minmax_data = pd.DataFrame(minmax.fit_transform(movie_data[['budget', 'revenue', 'runtime', 'release_date']]))

#standard_scaler = StandardScaler()
#standard_data = pd.DataFrame(standard_scaler.fit_transform(movie_data[['budget', 'revenue', 'runtime', 'release_date']]))
def normalize(column):
    upper = column.max()
    lower = column.min()
    y = (column - lower)/(upper-lower)
    return y

def feature_scaling(column, df):
    cube_root = df[column]**(1/3)
    cube_norm = normalize(cube_root)
    df[column] = cube_norm
    return df

columns_to_scale = ['budget', 'revenue', 'runtime', 'release_date']
for column in columns_to_scale:
    movie_data = feature_scaling(column, movie_data)

In [44]:
print(movie_data.iloc[:, :4].median())
movie_data.head()

budget          0.526473
revenue         0.427197
runtime         0.594727
release_date    0.804338
dtype: float64


budget   revenue   runtime  release_date  lead_male  lead_female  \
0  0.887693  0.853630  0.834416      0.661474          1            0   
1  1.000000  0.840522  0.669359      0.680071          1            0   
2  0.977222  0.770174  0.491777      0.819451          1            0   
3  0.988742  0.892533  0.642789      0.760979          1            0   
4  0.463148  0.513987  0.695324      0.805871          1            0   

   lead2_male  lead2_female  Action  Adventure  Animation  Comedy  Crime  \
0           1             0       0          0          0       0      0   
1           0             1       1          1          0       0      0   
2           1             0       1          0          0       1      1   
3           1             0       1          1          0       0      0   
4           1             0       1          1          0       0      0   

   Documentary  Drama  Family  Fantasy  Foreign  History  Horror  Music  \
0            0      1       0        0        0        0       0      0   
1            0      0       0        1        0        0       0      0   
2            0      0       0        0        0        0       0      0   
3            0      0       0        1        0        0       0      0   
4            0      1       0        0        0        0       1      0   

   Mystery  Romance  Science Fiction  TV Movie  Thriller  War  Western  1960s  \
0        0        1                0         0         0    0        0      0   
1        0        1                0         0         0    0        0      0   
2        0        0                0         0         1    0        0      0   
3        0        0                0         0         0    0        0      0   
4        0        0                1         0         0    0        0      0   

   1970s  1980s  19th century  3d  adoption  adultery  adventure  \
0      0      0             0   1         0         0          0   
1      0      0             0   0         0         0          0   
2      0      0             0   0         0         0          0   
3      0      0             0   0         0         0          0   
4      0      0             0   0         0         0          0   

   aftercreditsstinger  afterlife  agent  airplane  airport  alcohol  \
0                    0          0      0         0        0        0   
1                    0          0      0         0        0        0   
2                    0          0      0         0        0        0   
3                    0          0      0         0        0        0   
4                    0          0      0         0        0        0   

   alcoholic  alcoholism  alien  alien invasion  american football  amnesia  \
0          0           0      0               0                  0        0   
1          0           0      0               0                  0        0   
2          0           0      0               0                  0        0   
3          0           0      0               0                  0        0   
4          0           0      0               0                  0        0   

   android  animation  anti hero  army  artificial intelligence  assassin  \
0        0          0          0     0                        0         0   
1        0          0          0     0                        0         0   
2        0          0          0     0                        0         0   
3        0          0          0     0                        0         0   
4        0          0          0     0                        0         0   

   assassination  astronaut  australia  author  baby  bank  bar  baseball  \
0              0          0          0       0     0     0    0         0   
1              0          0          0       0     0     0    0         0   
2              0          0          0       0     0     0    0         0   
3              0          0          0       0     0     0    0         0   
4     

In [45]:
Y = (movie_data['revenue'])
X = movie_data.drop(['revenue'], axis = 'columns')
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.33, random_state=42)

In [89]:
clf = SVR(kernel = 'linear', C=0.01, epsilon=0.002)
clf.fit(X_train, Y_train)
pred_clf = clf.predict(X_test)
coefs_clf = clf.coef_
print('R^2 for SVR with linear kernel:', clf.score(X_test, Y_test), r2_score(Y_test, pred_clf))

R^2 for SVR with linear kernel: 0.45443652941079093 0.45443652941079093


In [88]:
ridge = Ridge(alpha = 20)
ridge.fit(X_train, Y_train)
pred_ridge = ridge.predict(X_test)
coefs_ridge = ridge.coef_
print('R^2 for ridge regression:', ridge.score(X_test, Y_test), r2_score(Y_test, pred_ridge))

R^2 for ridge regression: 0.4758268223989934 0.4758268223989933


In [107]:
rand_for = RandomForestRegressor(n_estimators = 100)
rand_for.fit(X_train, Y_train)
pred_rand_for = rand_for.predict(X_test)
#coefs_ridge = rand_for.coef_
print('R^2 for random forest regressor:', r2_score(Y_test, pred_rand_for))

R^2 for random forest regressor: 0.45694840548946847


In [113]:
from xgboost import XGBRegressor
xgb = XGBRegressor(colsample_bytree= 0.6, gamma= 0.7, max_depth= 4, min_child_weight= 5,
                         subsample = 0.8, objective='reg:squarederror')
xgb.fit(X_train, Y_train)

pred_xgb = xgb.predict(X_test)
print('R^2 for random forest regressor:', r2_score(Y_test, pred_xgb))

R^2 for random forest regressor: 0.4240099751211869


In [112]:
from sklearn.neural_network import MLPRegressor
nn = MLPRegressor((500, 500), alpha = 0.1, verbose = True)
nn.fit(X_train, Y_train)
pred_nn = nn.predict(X_test)
#coefs_ridge = rand_for.coef_
print('R^2 for neural net:', r2_score(Y_test, pred_nn)) #0.3816737188287369

Iteration 1, loss = 0.36243766
Iteration 2, loss = 0.25503318
Iteration 3, loss = 0.19335810
Iteration 4, loss = 0.15001729
Iteration 5, loss = 0.11970988
Iteration 6, loss = 0.09814614
Iteration 7, loss = 0.08230087
Iteration 8, loss = 0.07022191
Iteration 9, loss = 0.06081704
Iteration 10, loss = 0.05329874
Iteration 11, loss = 0.04717780
Iteration 12, loss = 0.04221124
Iteration 13, loss = 0.03838592
Iteration 14, loss = 0.03499767
Iteration 15, loss = 0.03217128
Iteration 16, loss = 0.02947630
Iteration 17, loss = 0.02712933
Iteration 18, loss = 0.02501208
Iteration 19, loss = 0.02328741
Iteration 20, loss = 0.02164486
Iteration 21, loss = 0.02017905
Iteration 22, loss = 0.01886629
Iteration 23, loss = 0.01765355
Iteration 24, loss = 0.01642811
Iteration 25, loss = 0.01537475
Iteration 26, loss = 0.01440807
Iteration 27, loss = 0.01366371
Iteration 28, loss = 0.01287604
Iteration 29, loss = 0.01225088
Iteration 30, loss = 0.01186052
Iteration 31, loss = 0.01135557
Iteration 32, los